In [1]:
import numpy as np
import pandas as pd

In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# Read the parquet file
transactions_df = pd.read_parquet('indiv_transactions.parquet')
# Display the first few rows
print(transactions_df.head())

                                         CUST_CUSTNO  VALUEDATE  \
0  00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608... 2023-08-05   
1  00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608... 2023-08-06   
2  00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608... 2023-08-10   
3  00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608... 2023-08-11   
4  00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608... 2023-08-13   

   TOTAL_RECEIVED  TOTAL_SENT  TOTAL_ABSOLUTE  
0          1500.0      -300.0          1800.0  
1             0.0     -1000.0          1000.0  
2         12717.6     -2063.0         14780.6  
3             0.0     -3439.0          3439.0  
4             0.0     -2500.0          2500.0  


In [3]:
# Count the number of transactions per account
transaction_counts = transactions_df.groupby('CUST_CUSTNO').size().reset_index(name='TRANSACTION_COUNT')

# Display the result
transaction_counts

,CUST_CUSTNO,TRANSACTION_COUNT
0,00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608...,94
1,00002bfa6df6d71eb18c1669be8d381641529f0dcdbb2a...,8
2,0000398a220cd0cfa550b1b352bc3928ac970e935b5f44...,1
3,00004da252a8699874230b3b45cd8af120225da77aaa91...,1
4,000054efc404498dc560a7562c62dfa408fe2088001d06...,3
...,...,...
968177,ffffaa126887038ed403e49535cebc14cc4f1c1a28e62c...,143
968178,ffffe6ffe4982025013118d59589d6f01c57e766082750...,6
968179,ffffecac7ebed6cb3c63bf4be888d7cff04f68318c9129...,19
968180,fffff3420218b811433e791f51df6f66b256dd49b35077...,31


In [4]:
# Display summary statistics
print(transaction_counts['TRANSACTION_COUNT'].describe())

count    968182.000000
mean         31.483185
std          44.817480
min           1.000000
25%           2.000000
50%           8.000000
75%          44.000000
max         180.000000
Name: TRANSACTION_COUNT, dtype: float64


In [5]:
MIN_TRANSACTIONS = 10 # threshold

# Get accounts with at least MIN_TRANSACTIONS
valid_accounts = transaction_counts[transaction_counts['TRANSACTION_COUNT'] >= MIN_TRANSACTIONS]['CUST_CUSTNO']

# Filter transactions
filtered_transactions = transactions_df[transactions_df['CUST_CUSTNO'].isin(valid_accounts)]

# Save filtered data
filtered_transactions.to_parquet('filtered_transactions.parquet', index=False)

# Display results
print(f"Filtered transactions: {filtered_transactions.shape[0]} rows")
print(f"Remaining accounts: {filtered_transactions['CUST_CUSTNO'].nunique()}")


Filtered transactions: 28997435 rows
Remaining accounts: 459915


In [4]:
# Rule-based anomalies
rule_15_anomalies = filtered_transactions[
    (filtered_transactions['TOTAL_RECEIVED'] >= 6000000) &
    (filtered_transactions['TOTAL_SENT'].abs() >= 0.95 * filtered_transactions['TOTAL_RECEIVED'])
]
rule_15_anomalies

,CUST_CUSTNO,VALUEDATE,TOTAL_RECEIVED,TOTAL_SENT,TOTAL_ABSOLUTE
702,0002270df70734f8d04c42004d9995e8b97fe3ff9de575...,2023-11-08,6925150.00,-6924000.00,13849150.00
2170,0004abe134ca2b7af66656ecc3232a24a1167e71aa2bd9...,2023-09-23,8000493.15,-8000008.22,16000501.37
4095,000975b0ee367856502b174994023821120d5730b3fcd3...,2023-08-22,10469355.99,-10051700.46,20521056.45
4102,000975b0ee367856502b174994023821120d5730b3fcd3...,2023-08-29,7179280.10,-7513252.10,14692532.20
4142,000975b0ee367856502b174994023821120d5730b3fcd3...,2023-10-10,7609562.68,-9133854.50,16743417.18
...,...,...,...,...,...
28979127,ffdc5cead1c3798acfabfbc6eb7646e4b5d9e540f35ee3...,2023-09-30,12854001.29,-12512900.03,25366901.32
28985946,ffe85c70096ca3b80d36d42577c9c12eb0ed1ad9508851...,2023-09-19,6012400.00,-6012400.00,12024800.00
28988045,ffecb0aaab47e4d92ac4176cdee0c3d9dd4617216035c8...,2023-08-31,6242683.78,-6231263.79,12473947.57
28988705,ffed5808f5090ac8cfa75f590ff0874f5787cde1784b40...,2023-12-29,9738344.66,-11274259.32,21012603.98


In [3]:
# Read the parquet file
filtered_transactions = pd.read_parquet('filtered_transactions.parquet')
# Display the first few rows
print(filtered_transactions.head())


                                         CUST_CUSTNO  VALUEDATE  \
0  00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608... 2023-08-05   
1  00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608... 2023-08-06   
2  00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608... 2023-08-10   
3  00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608... 2023-08-11   
4  00001fd6f1d8852373810b5402eb3ae6adbd610d9d3608... 2023-08-13   

   TOTAL_RECEIVED  TOTAL_SENT  TOTAL_ABSOLUTE  
0          1500.0      -300.0          1800.0  
1             0.0     -1000.0          1000.0  
2         12717.6     -2063.0         14780.6  
3             0.0     -3439.0          3439.0  
4             0.0     -2500.0          2500.0  


In [20]:
# Step 1: Get unique accounts with rule 15 anomalies
unique_anomalous_accounts = rule_15_anomalies['CUST_CUSTNO'].unique()

# Step 2: Randomly select 100 accounts (or all if fewer than 1000 exist)
random_anomalous_accounts = np.random.choice(unique_anomalous_accounts, size=min(1000, len(unique_anomalous_accounts)), replace=False)

# Step 3: Filter anomalies for selected accounts
sample_anomalies = rule_15_anomalies[rule_15_anomalies['CUST_CUSTNO'].isin(random_anomalous_accounts)]

# Step 4: Retrieve full historical transactions for selected accounts
sample_initial_data = filtered_transactions[filtered_transactions['CUST_CUSTNO'].isin(random_anomalous_accounts)]

# Display summary
print(f"Total selected accounts: {len(random_anomalous_accounts)}")
print(f"Total anomalies for selected accounts: {sample_anomalies.shape[0]}")
print(f"Total transactions for selected accounts: {sample_initial_data.shape[0]}")

# Optionally, preview the selected transactions and anomalies
print("\nSample transactions from selected accounts:")
print(sample_initial_data.head()) 

print("\nSample anomalies from selected accounts:")
print(sample_anomalies.head())


Total selected accounts: 1000
Total anomalies for selected accounts: 2355
Total transactions for selected accounts: 59375

Sample transactions from selected accounts:
                                            CUST_CUSTNO  VALUEDATE  \
2159  0004abe134ca2b7af66656ecc3232a24a1167e71aa2bd9... 2023-08-03   
2160  0004abe134ca2b7af66656ecc3232a24a1167e71aa2bd9... 2023-08-06   
2161  0004abe134ca2b7af66656ecc3232a24a1167e71aa2bd9... 2023-08-07   
2162  0004abe134ca2b7af66656ecc3232a24a1167e71aa2bd9... 2023-08-12   
2163  0004abe134ca2b7af66656ecc3232a24a1167e71aa2bd9... 2023-08-13   

      TOTAL_RECEIVED  TOTAL_SENT  TOTAL_ABSOLUTE  
2159      2111765.34 -2075882.63      4187647.97  
2160       118640.55  -112609.63       231250.18  
2161       242683.00  -243561.22       486244.22  
2162       600000.00  -300000.00       900000.00  
2163       105355.62  -100000.01       205355.63  

Sample anomalies from selected accounts:
                                             CUST_CUSTNO  VALUED

In [21]:
# Save the filtered initial transaction data to a Parquet file
sample_initial_data.to_parquet('sample_initial_data_1000.parquet', index=False)

# Save the anomaly data to a Parquet file
sample_anomalies.to_parquet('sample_anomalies_1000.parquet', index=False)

# Display confirmation message
print("Initial data and anomaly data have been saved to Parquet files.")


Initial data and anomaly data have been saved to Parquet files.


In [22]:
# Group the filtered transactions by account number and see the transactions for each account
transactions_by_account = sample_initial_data.groupby('CUST_CUSTNO').size().reset_index(name='TRANSACTION_COUNT')

# Optionally, display a few example accounts with their transaction counts
print("Transaction counts for each account:")
print(transactions_by_account)

# If you want to display the full list of accounts and their transaction counts
print(f"Total accounts in the dataset: {transactions_by_account.shape[0]}")


Transaction counts for each account:
                                           CUST_CUSTNO  TRANSACTION_COUNT
0    0004abe134ca2b7af66656ecc3232a24a1167e71aa2bd9...                 30
1    008305ae484962df097f666beadd0aaf025cfbc520151c...                 10
2    0087f4edecf2fe0053534d5682dc1efaa74a4f13491317...                 40
3    0095f2e50e917929b517470e2820afb537066e68b9eae2...                 56
4    0098e2623c5dcfb583cfcbf9aca11e08abc76f1c019ded...                168
..                                                 ...                ...
995  fefd988c4647b72e345ce2d8c4e7e5524fc840b30f6431...                 85
996  ff0136dc34c40d735c12e60c9d77cec4b7707ef67fba06...                 22
997  ff74521fb792c5f02c2d0b28f39bc230adf92c7b3f557b...                123
998  ff82d366b0d59f4d5a8474e7ee917231b7bfba62505cf1...                 69
999  ffed5808f5090ac8cfa75f590ff0874f5787cde1784b40...                163

[1000 rows x 2 columns]
Total accounts in the dataset: 1000


In [6]:
# Step 1: Compute anomaly ratio for each account
anomaly_counts = rule_15_anomalies['CUST_CUSTNO'].value_counts()
total_counts = filtered_transactions['CUST_CUSTNO'].value_counts()

# Create a dataframe for analysis
account_stats = pd.DataFrame({
    'total_transactions': total_counts,
    'anomaly_transactions': anomaly_counts
}).fillna(0)  # Fill missing anomaly counts with 0

# Calculate the anomaly ratio
account_stats['anomaly_ratio'] = account_stats['anomaly_transactions'] / account_stats['total_transactions']

# Step 2: Sort by anomaly ratio and filter eligible accounts
high_anomaly_accounts = account_stats.sort_values(by='anomaly_ratio', ascending=False)

# Ensure we have enough accounts before sampling
num_high_anomaly = min(500, len(high_anomaly_accounts))  # Avoid index errors
top_high_anomaly = high_anomaly_accounts.head(num_high_anomaly).index  # Most anomalous

# Step 3: Stratify remaining accounts by total transactions
remaining_accounts = high_anomaly_accounts.iloc[num_high_anomaly:]
remaining_accounts['bucket'] = pd.qcut(remaining_accounts['total_transactions'], q=5, duplicates='drop')

# Sample 500 accounts evenly across transaction volume
random_500_other = (
    remaining_accounts
    .groupby('bucket', group_keys=False)
    .apply(lambda x: x.sample(n=min(500 // remaining_accounts['bucket'].nunique(), len(x)), replace=False))
    .index
)

# Combine selections
selected_accounts = np.concatenate((top_high_anomaly, random_500_other))

# Step 4: Filter anomalies and transactions for selected accounts
sample_anomalies = rule_15_anomalies[rule_15_anomalies['CUST_CUSTNO'].isin(selected_accounts)]
sample_initial_data = filtered_transactions[filtered_transactions['CUST_CUSTNO'].isin(selected_accounts)]

# Display summary
print(f"Total selected accounts: {len(selected_accounts)}")
print(f"Total anomalies for selected accounts: {sample_anomalies.shape[0]}")
print(f"Total transactions for selected accounts: {sample_initial_data.shape[0]}")

# Preview data
print("\nSample transactions from selected accounts:")
print(sample_initial_data.head()) 

print("\nSample anomalies from selected accounts:")
print(sample_anomalies.head())


Total selected accounts: 1000
Total anomalies for selected accounts: 4811
Total transactions for selected accounts: 44232

Sample transactions from selected accounts:
                                             CUST_CUSTNO  VALUEDATE  \
36488  004ce26effc86476b419e4484ba44cf6e2ee79017b2857... 2023-08-10   
36489  004ce26effc86476b419e4484ba44cf6e2ee79017b2857... 2023-08-11   
36490  004ce26effc86476b419e4484ba44cf6e2ee79017b2857... 2023-08-21   
36491  004ce26effc86476b419e4484ba44cf6e2ee79017b2857... 2023-08-31   
36492  004ce26effc86476b419e4484ba44cf6e2ee79017b2857... 2023-09-21   

       TOTAL_RECEIVED  TOTAL_SENT  TOTAL_ABSOLUTE  
36488          3000.0         0.0          3000.0  
36489             0.0     -1000.0          1000.0  
36490             0.0     -2000.0          2000.0  
36491             0.1         0.0             0.1  
36492        607800.0   -707800.0       1315600.0  

Sample anomalies from selected accounts:
                                             CUST_CU

In [7]:
# Save the filtered initial transaction data to a Parquet file
sample_initial_data.to_parquet('data_mix_1000.parquet', index=False)

# Save the anomaly data to a Parquet file
sample_anomalies.to_parquet('anomaly_mix_1000.parquet', index=False)

# Display confirmation message
print("Initial data and anomaly data have been saved to Parquet files.")


Initial data and anomaly data have been saved to Parquet files.


In [ ]:
# Test3

In [5]:
# Step 1: Compute anomaly ratio for each account
anomaly_counts = rule_15_anomalies['CUST_CUSTNO'].value_counts()
total_counts = filtered_transactions['CUST_CUSTNO'].value_counts()

# Step 2: Create a dataframe with total transactions, anomaly transactions, and anomaly ratio
account_stats = pd.DataFrame({
    'total_transactions': total_counts,
    'anomaly_transactions': anomaly_counts
}).fillna(0)  # Fill missing anomaly counts with 0

account_stats['anomaly_ratio'] = account_stats['anomaly_transactions'] / account_stats['total_transactions']

# Step 3: Select the top 1000 accounts with the highest anomaly ratio
top_1000_high_anomaly = account_stats.sort_values(by='anomaly_ratio', ascending=False).head(1000)

# Step 4: Extract the selected account numbers
selected_accounts = top_1000_high_anomaly.index

# Step 5: Filter transactions and anomalies for selected accounts
sample_anomalies = rule_15_anomalies.loc[rule_15_anomalies['CUST_CUSTNO'].isin(selected_accounts)]
sample_initial_data = filtered_transactions.loc[filtered_transactions['CUST_CUSTNO'].isin(selected_accounts)]

# Step 6: Save to Parquet for performance optimization
sample_anomalies.to_parquet("top_1000_anomaly_ratio_anomalies.parquet", index=False)
sample_initial_data.to_parquet("top_1000_anomaly_ratio_transactions.parquet", index=False)

# Display summary
print(f"Total selected accounts: {len(selected_accounts)}")
print(f"Total anomalies for selected accounts: {sample_anomalies.shape[0]}")
print(f"Total transactions for selected accounts: {sample_initial_data.shape[0]}")

#  preview the selected transactions and anomalies
print("\nTop 10 selected accounts with highest anomaly ratios:")
print(top_1000_high_anomaly.head(10))

print("\nSample transactions from selected accounts:")
print(sample_initial_data.head())

print("\nSample anomalies from selected accounts:")
print(sample_anomalies.head())


Total selected accounts: 1000
Total anomalies for selected accounts: 7156
Total transactions for selected accounts: 25007

Top 10 selected accounts with highest anomaly ratios:
                                                    total_transactions  \
CUST_CUSTNO                                                              
dff3e1c2b30e16cb56914219f4cf25c5034ac1e89a020df...                  10   
a19234074027bafa467dc4555031410fb1f2d018dd60724...                  10   
86f58c1386011257a535908871bb5670241e7126299087c...                  99   
4aa39702af3449bfc73a1cb0b5db1af3153a1f3cc068fd6...                  13   
9241f22a91b2d0bb0a04ca2414692dd8e54983709915607...                  17   
93b3ca884b931a88e5f8127bc323a5d06b90b217cc16740...                  11   
aa5ad0cfb42956d33400214d37c9c997f6b37fcdc4ac428...                  22   
a2537bf938d7cbbe96a6110c87bf1cd631d5f7ccdc62ccf...                  10   
55c3519ba92b64e98465c5a042e5af669965eb0c12c8bfc...                  10   
effb53965

In [6]:
# Save the filtered initial transaction data to a Parquet file
sample_initial_data.to_parquet('data_best_1000.parquet', index=False)

# Save the anomaly data to a Parquet file
sample_anomalies.to_parquet('anomaly_best_1000.parquet', index=False)

# Display confirmation message
print("Initial data and anomaly data have been saved to Parquet files.")

Initial data and anomaly data have been saved to Parquet files.
